## 多个 feas_iti 且最后一趟列车 id 相同的乘客

|           |     rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
| :-------- | ------: | -----: | ---------: | -----: | -------: | -------: | --------: |
| 93194730  | 1525794 |      2 | 1076103701 |      2 | 10301630 |    65071 |     65670 |
| 8610761   |  119014 |      6 | 1101102701 |      3 | 11002833 |    30045 |     30791 |
| 18578209  |  230428 |      6 | 1053108503 |      2 | 10401849 |    29715 |     30255 |
| 72855273  | 1321497 |      3 | 1110112802 |      1 | 10401844 |    60803 |     61537 |
| 105488240 | 1781873 |      2 | 1028106602 |      2 | 10401738 |    69396 |     69625 |
| 90908613  | 1485498 |      2 | 1023112302 |      2 | 10702370 |    64374 |     64780 |
| 91388272  | 1494286 |      1 | 1104111501 |      2 | 10702247 |    65070 |     65278 |
| 53621559  |  957712 |      6 | 1121102503 |      3 | 10401786 |    49174 |     50141 |
| 19767473  |  243641 |      1 | 1081103901 |      2 | 10702323 |    29509 |     31141 |
| 107978876 | 1880069 |      2 | 1129107002 |      1 | 10201226 |    73278 |     74154 |
| 106615427 | 1817923 |      1 | 1027110801 |      2 | 10301362 |    71471 |     73514 |
| 76737269  | 1359494 |      1 | 1099102101 |      2 | 10301421 |    64071 |     65266 |
| 100422001 | 1661221 |      5 | 1061108702 |      1 | 10100270 |    66558 |     66886 |
| 2408620   |   41409 |      2 | 1111106502 |      1 | 10401886 |    25727 |     25836 |
| 101105912 | 1675451 |      1 | 1110101301 |      1 | 10401914 |    66944 |     67031 |
| 92513072  | 1514419 |      3 | 1097110302 |      3 | 10200839 |    65302 |     65408 |
| 81121188  | 1412555 |      1 | 1061109504 |      2 | 10201199 |    63371 |     63480 |
| 108069499 | 1886032 |      1 | 1032108201 |      1 | 10200950 |    73568 |     75192 |
| 73737366  | 1331466 |      1 | 1086110901 |      2 | 10702414 |    61616 |     62107 |
| 48046191  |  811513 |      3 | 1076103602 |      3 | 10301265 |    42747 |     43084 |


In [1]:
from src import config
from src.utils import *
from src.globals import *

config.load_config()

assigned = read_all(config.CONFIG["results"]["assigned"], show_timer=False)
assigned

[INFO] Reading 1 versioned files: results\trajectory\assigned.pkl


,rid,iti_id,path_id,seg_id,train_id,board_ts,alight_ts
0,132,1,1127110101,1,10200773,22740,23821
1,284,1,1027103801,1,11002637,20292,21355
2,308,1,1132107101,1,10401755,22659,24066
3,312,1,1101103301,1,10200756,20219,22237
4,313,1,1101103301,1,10200756,20219,22237
...,...,...,...,...,...,...,...
906237,2047044,1,1055100801,1,10402102,85925,86264
906238,2047046,1,1091111301,1,10401899,85983,86200
906239,2047049,1,1025111301,1,10401899,86095,86200
906240,2047059,1,1114105701,1,10702494,86172,86255


In [2]:
from src.timetable import *

train_id = np.random.choice(get_tt()[:, 0], size=1)[0]
assigned_arr = assigned[assigned["train_id"] == train_id][["board_ts", "alight_ts"]].values
calculate_train_load_profile(train_id, assigned_arr)

[INFO] Reading file: data\TT.pkl


array([[10143, 69561, 10142, 69671,     2],
       [10142, 69706, 10141, 69796,     5],
       [10141, 69846, 10140, 69963,     6],
       [10140, 69998, 10139, 70092,     7],
       [10139, 70127, 10138, 70210,     9],
       [10138, 70247, 10137, 70328,    14],
       [10137, 70365, 10136, 70472,    30],
       [10136, 70507, 10135, 70579,    33],
       [10135, 70614, 10134, 70684,    37],
       [10134, 70719, 10133, 70809,    45],
       [10133, 70846, 10132, 70944,    50],
       [10132, 70989, 10131, 71073,    46],
       [10131, 71110, 10130, 71190,    52],
       [10130, 71227, 10129, 71303,    43],
       [10129, 71345, 10128, 71424,    48],
       [10128, 71462, 10127, 71531,    39],
       [10127, 71566, 10126, 71635,    40],
       [10126, 71685, 10125, 71766,    46],
       [10125, 71808, 10124, 71881,    41],
       [10124, 71916, 10123, 72010,    32],
       [10123, 72045, 10122, 72133,    24],
       [10122, 72173, 10121, 72298,    19],
       [10121, 72333, 10120, 724

In [10]:
train_ids = np.random.choice(get_tt()[:, 0], size=100)
res: dict[int, np.ndarray] = {}
for train_id, board_counts in assigned["train_id"].value_counts().items():
    cap, cap_max = 500, 800
    if board_counts < cap:  # all boarded less than normal capacity, skip
        continue

    # ['sta1_nid', 'dep_ts','sta2_nid', 'arr_ts', 'load']
    load_arr = calculate_train_load_profile(
        train_id, 
        assigned[assigned["train_id"] == train_id][["board_ts", "alight_ts"]].values
        )
    if np.all(load_arr[:, -1] < cap):  # section passengers less than capacity, skip
        continue

    if np.any(load_arr[:, -1] > cap_max):
        print(
            f"\033[33m[WARNING] Train {train_id} exceeds max capacity!\033[0m")

    res[train_id] = load_arr[load_arr[:, -1] > cap]

res

[WARNING] Train 10100135 exceeds max capacity!


{10200732: array([[10248, 28986, 10247, 29075,   567],
        [10247, 29110, 10246, 29301,   606],
        [10246, 29351, 10245, 29460,   630],
        [10245, 29497, 10244, 29582,   657],
        [10244, 29617, 10243, 29727,   662],
        [10243, 29765, 10242, 29844,   669],
        [10242, 29884, 10241, 30003,   575],
        [10241, 30038, 10240, 30121,   646],
        [10240, 30156, 10239, 30222,   658],
        [10239, 30259, 10238, 30366,   627],
        [10238, 30404, 10237, 30479,   587]], dtype=int64),
 10200956: array([[10245, 29137, 10244, 29222,   535],
        [10244, 29257, 10243, 29367,   543],
        [10243, 29405, 10242, 29484,   557],
        [10242, 29524, 10241, 29643,   544],
        [10241, 29678, 10240, 29761,   605],
        [10240, 29796, 10239, 29862,   622],
        [10239, 29899, 10238, 30006,   620],
        [10238, 30044, 10237, 30119,   557]], dtype=int64),
 10200799: array([[10246, 30251, 10245, 30360,   510],
        [10245, 30397, 10244, 30482,   5